In [36]:
import sympy as smp
import itertools as it
from sympy.tensor.array.dense_ndim_array import MutableDenseNDimArray
import numpy as np
import re
from src.RelativisticPy.general_relativity.direct_computations.gr_computations import GrComputations
from src.RelativisticPy.general_relativity.string_to_sympy.sympy_parser import SympyParser
from src.RelativisticPy.general_relativity.tensor.gr_tensor import GrTensor
from src.RelativisticPy.general_relativity.tensor.tensor import Tensor
from src.RelativisticPy.general_relativity.index.gr_index import GrIndex
from src.RelativisticPy.general_relativity.indices.gr_indices import GrIndices
from src.RelativisticPy.general_relativity.index.index import Index
from src.RelativisticPy.general_relativity.tensor_operation.index_product import IndexProduct
from src.RelativisticPy.general_relativity.tensor_operation.gr_tensor_product import GrTensorProduct
from src.RelativisticPy.math_json.to_math_json.string_to_math_json.interpreter import Interpreter
from src.RelativisticPy.math_json.to_math_json.string_to_math_json.lexer import Lexer
from src.RelativisticPy.math_json.to_math_json.string_to_math_json.parser_ import Parser

In [38]:
SympyParser("[[-A,0,0,0],[0,B,0,0],[0,0,r**2,0],[0,0,0,r**2*smp.sin(theta)**2]]").convertToSympyObject()
m, sigma, C_0, alpha, beta, r_s, M, t, r, theta, phi, tau, x, y, z, l, E = smp.symbols('m sigma C_0 alpha beta r_s M t r theta phi tau x y z l E', float = True)

t_p, r_p, theta_p, phi_p = smp.symbols('t_p r_p theta_p phi_p', cls = smp.Function)

t_p = t_p(tau)
r_p = r_p(tau)
theta_p = theta_p(tau)
phi_p = phi_p(tau)


A , B = smp.symbols('A B', cls = smp.Function)
A = A(r)
B = B(r)

Metric = smp.MutableDenseNDimArray([[-A,0,0,0],[0,B,0,0],[0,0,r**2,0],[0,0,0,r**2*smp.sin(theta)**2]])
InvMetric = smp.MutableDenseNDimArray([[-1/A,0,0,0],[0,1/B,0,0],[0,0,1/r**2,0],[0,0,0,1/(r**2*smp.sin(theta)**2)]])
Base = smp.Array([t,r,theta,phi])
Dimention = 4
GR_General_Metric = GrComputations(Metric, Base)
MS = smp.Array([[-(1 - (r_s)/(r_p)),0,0,0],[0,1/(1 - (r_s)/(r_p)),0,0],[0,0,r_p**2,0],[0,0,0,r_p**2*smp.sin(theta_p)**2]])

Ricci = GR_General_Metric.Ricci()
RimannCov = GR_General_Metric.Riemann0000()

In [50]:
class Workspace:

    def __init__(self, tensor_string):
        self.database = {'G' : { 'Components' : Metric , 'Basis' : Base},
                         'Ginv' : { 'Components' : InvMetric , 'Basis' : Base},
                         'Ricci' : { 'Components' : Ricci , 'Basis' : Base},
                         'CovRiemann' : { 'Components' : RimannCov , 'Basis' : Base}}
        self.tensor_string = tensor_string
        self.tensor_object = GrTensor(self.database[self.representation_one()['name']]['Components'], GrIndices(self.representation_one()['indices'], self.database[self.representation_one()['name']]['Basis']))

    def representation_one(self):
        string = self.tensor_string
        name = re.match('([a-zA-Z]+)', string).group()
        indices = string.replace(name, '')
        return {'name' : name, 'indices' : indices}

class ComputeMathJson:
    def __init__(self, x):
        database = {}
        self.operations = {"ADD" : lambda List : ComputeMathJson(List[0])+ComputeMathJson(List[1]),
                           "MUL" : lambda List : ComputeMathJson(List[0])*ComputeMathJson(List[1]),
                           "MINUS" : lambda List : ComputeMathJson(List[0])-ComputeMathJson(List[1]),
                           "DIV" : lambda List : ComputeMathJson(List[0])/ComputeMathJson(List[1]),
                           "DIFF" : lambda List : smp.diff(List[0], List[1]),
                           "INT" : lambda List : smp.integrate(List[0], List[1]),
                           "EQUALS" : lambda List : database.update({List[0] : ComputeMathJson(List[1]).x}),
                           "NUMBER_INT" : lambda x : int(x),
                           "POW" : lambda List : ComputeMathJson(List[0])**ComputeMathJson(List[1]),
                           "TENSOR" : lambda x : Workspace(x).tensor_object,
                           "sympy_array" : lambda x : MutableDenseNDimArray(self.Generate(x)),
                           "sympy_variable" : lambda x : smp.symbols(x),
                           "sympy_function" : lambda List : smp.symbols(List[0] , cls = smp.Function)(*tuple([ComputeMathJson(List[1][i]).x for i in range(len(List[1]))]))}
        
        if isinstance(x, dict):
            if [i for i in x][0] == "DIFF":
                self.x = self.operations["DIFF"](x["DIFF"])
            elif [i for i in x][0] == "sympy_variable":
                self.x = self.operations["sympy_variable"](x["sympy_variable"])
            elif [i for i in x][0] == "sympy_function":
                self.x = self.operations["sympy_function"](x["sympy_function"])
            elif [i for i in x][0] == "sympy_array":
                self.x = self.operations["sympy_array"](x["sympy_array"])
            elif [i for i in x][0] == "INT":
                self.x = self.operations["INT"](x["INT"])
            elif [i for i in x][0] == "POW":
                self.x = self.operations["POW"](x["POW"])
            elif [i for i in x][0] == "NUMBER_INT":
                self.x = self.operations["NUMBER_INT"](x["NUMBER_INT"])
            elif [i for i in x][0] == "TENSOR":
                self.x = self.operations["TENSOR"](x["TENSOR"])
            else:
                self.x = self.operations[[i for i in x][0]](x[[i for i in x][0]])
        else:
            self.x = x
            
    def Generate(self, arr):
        IterableComponentsForArray = lambda Array : list(it.product(np.arange(np.array(Array).shape[0]), repeat = len(np.array(Array).shape)))
        new = np.array(arr)
        for i in IterableComponentsForArray(arr):
            new[i] = ComputeMathJson(np.array(arr)[i]).x
        return new
            
    def __add__(self, other):
        return self.x + other.x
    
    def __mul__(self, other):
        return self.x*other.x
    
    def __sub__(self, other):
        return self.x-other.x
    
    def __truediv__(self, other):
        return self.x/other.x

    def __pow__(self, other):
        return self.x**other.x

In [61]:
text = '12*3'
lexerr = Lexer(text)
tokens = lexerr.generate_tokens()
parser = Parser(tokens)
tree = parser.parse()
inter = Interpreter()
math_json = inter.visit(tree)
math_json.dict

{'MUL': [{'NUMBER_INT': '12'}, {'NUMBER_INT': '3'}]}

In [63]:
ComputeMathJson(math_json.dict).x

36

In [149]:
input_box = 'G^{a}^{b}*G_{b}_{d}'
ComputeMathJson(StringToMathJson(input_box).calculate().objectJson)

AttributeError: 'GrTensorProduct' object has no attribute 'return_tensor'

In [114]:
TensorStringRepresentations('G_{a}_{b}').tensor_object

In [96]:
TensorA = 'Metric_{Mu}_{b}'

name = re.match('([a-zA-Z]+)', TensorA).group()

TensorA.replace(name, '')

'_{Mu}_{b}'